In [1]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import time
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.inspection import partial_dependence
import seaborn as sns
random_seed=251
random.seed(random_seed)
np.random.seed(random_seed)

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)

In [2]:
# moving_window=12

df= pd.read_csv('D:/istb_4/ISTB4_fillcombdata2.csv')

# df_x_m=df1.groupby(pd.RangeIndex(len(df1)) // 12).mean()
# df_x_m=df1.rolling(moving_window).mean()
# df_ready=df_x_m.dropna()
# df=df_ready.dropna()

In [3]:
# df['time_index']=list(range (len(df.Time)))
# df['month_index']=pd.DatetimeIndex(df['Time']).month
# df['hour_index']=pd.DatetimeIndex(df['Time']).hour
# df['day_index']=pd.DatetimeIndex(df['Time']).weekday
# df['week_index']=pd.DatetimeIndex(df['Time']).isocalendar()['week'].to_numpy()

In [4]:
def norm(df1,df2):
    df1_norm=(df1-df2.min())/(df2.max()-df2.min())
    return df1_norm

In [5]:

Cl_y = df['Chlorine.ppm.']
Cl_x =df[['pH','Conductivity.uS.','Temperature.C.','ORP.mV.','ISTB4.Sum']]#,'time_index','month_index','week_index','day_index','hour_index']]

In [ ]:
"'  shuffle'"
dx =Cl_x[:-1]
dy =Cl_y[1:]


X_train,X_test,y_train,y_test=train_test_split(dx,dy ,test_size=0.2, random_state=random_seed)
# X_train,X_cv,y_train,y_cv=train_test_split(X_traincv,y_traincv ,test_size=0.2, random_state=random_seed)

X_train.shape, y_train.shape,X_test.shape, y_test.shape


In [ ]:
'time order-all time'
# X_train =norm(Cl_x[:55000],Cl_x[:55000])
# y_train =Cl_y[1:55001]

# X_test =norm(Cl_x[55000:-1],Cl_x[:55000])
# y_test =Cl_y[55001:]
# X_train.shape, y_train.shape,X_test.shape, y_test.shape

In [ ]:
# 'time order-a slice of time'
# X_train =norm(Cl_x[40000:48000], Cl_x[40000:48000])
# y_train =Cl_y[40001:48001]

# X_test=norm(Cl_x[48000:49999], Cl_x[40000:48000])
# y_test =Cl_y[48001:50000]
# X_train.shape, y_train.shape,X_test.shape, y_test.shape

In [ ]:


rfm = RandomForestRegressor(n_jobs=-1,n_estimators=1000,random_state=random_seed,max_features=4,min_samples_leaf=20)
rfm.fit(X_train,y_train)
vari_imp_mn=rfm.feature_importances_
yhte= rfm.predict(X_test)
yhtr=rfm.predict(X_train)
# yhcv=rfm.predict(X_cv)


In [ ]:
mean_squared_error (y_train,yhtr),mean_squared_error (y_test,yhte),#mean_squared_error (y_cv,yhcv)

In [ ]:
r2_score(y_train,yhtr),r2_score(y_test,yhte),#r2_score(y_cv,yhcv)

# permutation

In [ ]:

resultss = permutation_importance(rfm, X_train, y_train, n_repeats=10,
                                random_state=0)


In [ ]:
variables_name=['pH', 'Conductivity', 'Temperature', 'ORP', 'Occupancy']

forest_importances = pd.DataFrame(data=resultss.importances_mean, index=variables_name, columns=['permute'])
kkk_sort=forest_importances.sort_values('permute',ascending=False)



In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(5,3), dpi=200)

ax.bar(kkk_sort.index,kkk_sort['permute']/kkk_sort['permute'].sum())
ax.set_ylabel('Variable importance',fontsize=txt_size)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in',labelsize=label_size)
ax.yaxis.set_ticks_position('both')
ax.set_yticks(ticks=np.arange(0,0.45,.1),labels=['0','0.1','0.2','0.3','0.4'])
plt.show()

# PDP

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
PartialDependenceDisplay.from_estimator(rfm,X_train, [0, 1, 2,3,4],kind='both')

In [ ]:
txt_size=12
label_size=10
fig, ax=plt.subplots(nrows=2, ncols=3, sharey= True, figsize=(10,5), dpi=200)
variables_name=['pH', 'Conductivity (uS/cm)', 'Temperature ($^o$ C)', 'ORP (mV)', 'Occupancy']
axes=ax.flatten()
for i_inde in range (5):
    results = partial_dependence(rfm,X_train, [i_inde],grid_resolution=20000,percentiles=[0, 1])
    axes[i_inde].plot(np.array(results["values"]).flatten(),np.array(results["average"]).flatten(),'k')
    axes[i_inde].tick_params(axis='both', direction='in',labelsize=label_size) 
    axes[i_inde].set_xlabel(variables_name[i_inde], fontsize=txt_size)
    sns.rugplot(X_train.iloc[:,i_inde],ax=axes[i_inde],c='k', alpha=0.03,height=0.03)
axes[0].set_ylabel('Predicted Chlorine (ppm)', fontsize=txt_size)
axes[3].set_ylabel('Predicted Chlorine (ppm)', fontsize=txt_size)
fig.delaxes(axes[-1])
fig.tight_layout() 
plt.show()

# AUTOcorrelation of Xnp.arange(0,1, .2)

In [ ]:
txt_size=12
label_size=10
fig, ax=plt.subplots(nrows=2, ncols=3,figsize=(10,5), dpi=200)
name_origin=['pH','Conductivity.uS.','Temperature.C.','ORP.mV.','ISTB4.Sum','Chlorine.ppm.']
variables_name=['pH', 'Conductivity (uS/cm)', 'Temperature ($^o$ C)', 'ORP (mV)', 'Occupancy','Chlorine (ppm)']
axes=ax.flatten()
time_in8hour=np.array(list(range(1,96)))*5/60
for k_indx in range (len(name_origin)):
    auto=[pd.Series(df[name_origin[k_indx]]).autocorr(lag=i_xx) for i_xx in range (1,96)]
    axes[k_indx].plot(time_in8hour,np.array(auto), 'k','o-', markersize=2)
    axes[k_indx].set_yticks(ticks=np.arange(0,1.2, .2),labels=['0','0.2','0.4','0.6','0.8','1'])
    axes[k_indx].tick_params(direction='in',labelsize=label_size)
    axes[k_indx].set_xlabel('Lag (hours)', fontsize=txt_size)
    axes[k_indx].set_ylabel(variables_name[k_indx], fontsize=txt_size)
fig.tight_layout() 
plt.show()

# correlation

In [ ]:
name_origin=['pH','Conductivity.uS.','Temperature.C.','ORP.mV.','ISTB4.Sum','Chlorine.ppm.']

corr=np.corrcoef(df[name_origin], rowvar=False)

In [ ]:
corr

In [ ]:
txt_size=13
label_size=9
from matplotlib.colors import CenteredNorm

fig,axes=plt.subplots(nrows=1, ncols=1, sharey='all',figsize=(5,3),dpi=200)
axes =sns.heatmap(corr,cmap='bwr',square=True,linewidths=0.35, center=0,linecolor='white'
                  , cbar_kws={'label': 'Correlation coefficient'}
                 )# axes.invert_yaxis()

axes.set_xticks(ticks=np.arange(0.5,6.5, 1),labels=variables_name,rotation=90)
axes.set_yticks(ticks=np.arange(0.5,6.5, 1),labels=variables_name,rotation=0)
axes.tick_params(labelsize=label_size,length=2)


# variable IMPORTANCE

In [ ]:


variables_name=['pH', 'Conductivity', 'Temperature', 'ORP', 'Occupancy']

varplot=pd.DataFrame(data= plot_vari_imp, index=variables_name, columns=['gini'])
dfx = varplot.sort_values('gini',ascending=False)
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(5,3), dpi=200)

ax.bar(dfx.index,dfx['gini'])
ax.set_ylabel('Variable importance',fontsize=txt_size)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in',labelsize=label_size)
ax.yaxis.set_ticks_position('both')
# ax.set_xticks(variables_name, fontsize=label_size)
plt.show()

# RF zoom in time series

In [ ]:
test_tm=pd.DataFrame ( data= list(zip(y_test, yhte)), index =y_test.index, columns=['insitu','rf'])
test_sort=test_tm.sort_index()

In [ ]:
y_train.sort_index().index

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [ ]:
from matplotlib import rc
from matplotlib.patches import ConnectionPatch
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10


zoom_start=41000
zoom_end=42000
# fig, axes = plt.subplots(figsize=[10,4])
fig = plt.figure(figsize=(10, 5), dpi=200)
b_right = fig.add_subplot(2, 2, 4)
b_left= fig.add_subplot(2, 2, 3)
main = fig.add_subplot(2, 1, 1)
fig.subplots_adjust(hspace=.35)
# fig,axes=plt.subplots(nrows=1, ncols=1, sharey='all',figsize=(10,5),dpi=200)
# axes.plot(test_sort.index,test_sort.insitu,  color='k',label='Insitu')
main.plot(Cl_y.index, Cl_y,color='k',label='Insitu')
# main.scatter(y_train.sort_index().index, y_train.sort_index(),color='orange',marker='o',s=1,linestyle='-',label='Insitu')
main.plot(test_sort.index,test_sort.rf, linestyle='-',  color='b',label='RF test')

main.legend()
main.set_xlabel('Time step',fontsize=txt_size)
main.set_ylabel('Chlorine(ppm) ',fontsize=txt_size)
main.tick_params(axis='both', direction='in',labelsize=label_size) 





b_right.scatter(Cl_y.index, Cl_y,color='k',s=5,label='Insitu')
b_right.scatter(test_sort.index,test_sort.rf,color='b',marker='^',s=5,linestyle='-', label='RF test')
b_right.set_xlim(zoom_start,zoom_end)
b_left.scatter(y_test,yhte,  color='b', marker='s', s=2, alpha=0.5, label='RF')
b_left.plot(y_test,y_test,  color='k')


b_left.set_xlabel('Chlorine measured (ppm)', fontsize=txt_size)


b_left.set_ylabel('Chlorine predicted (ppm)', fontsize=txt_size)
b_left.text(0.5, 0.05, '$R^2=$' +str(round(r2_score(y_test,yhte),3))+'\n'+ '$RMSE=$'+str(round(mean_squared_error (y_test,yhte)**0.5,3))
            +'(ppm)'
            ,  transform=b_left.transAxes,color="k", fontsize=txt_size)

b_left.tick_params(labelsize=label_size)
b_left.tick_params(labelsize=label_size)
b_left.tick_params(axis='both', direction='in',labelsize=label_size) 
b_right.tick_params(axis='both', direction='in',labelsize=label_size) 



fig.add_artist(ConnectionPatch(
    xyA=(0, 1), coordsA=b_right.transAxes,
    xyB=(zoom_start, Cl_y[zoom_start]), coordsB=main.transData,
    color='r'
))

fig.add_artist(ConnectionPatch(
    xyA=(1, 1), coordsA=b_right.transAxes,
    xyB=(zoom_end, Cl_y[zoom_end]), coordsB=main.transData,
    color='r'
))
main.axvspan(zoom_start, zoom_end, facecolor='r', alpha=0.5,zorder=-100)
plt.tight_layout()

In [ ]:

# plt.scatter(np.array(list(range(len(yhte))))+55000, yhte,color='blue',s=3, label='RF', alpha=0.5)
# plt.scatter(np.array(list(range(len(yhte))))+55000, y_test, color='orange', s=3, label='In situ', alpha=0.5)
# plt.ylabel('Chlorin')
# plt.xlabel('time')
# plt.legend()
# plt.title('test data time series')

In [ ]:
# "' N time step ahead time order with chlorine'"
# step_list=list(range(1,96))
# step_list[0]=1
# r2_te_list=[]
# r2_tr_list=[]
# mse_tr=[]
# mse_te=[]
# for step in step_list:
#     # step=1
#     X_train =norm(Cl_x[:55000],Cl_x[:55000])
#     y_train =Cl_y[step:55000+step]

#     X_test =norm(Cl_x[55000:len(Cl_x)-step],Cl_x[:55000])
#     y_test =Cl_y[55000+step:]
#     rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=random_seed,max_features=5,min_samples_leaf=80)
#     rfm.fit(X_train,y_train)
#     vari_imp_mn=rfm.feature_importances_
#     hpa_yh= rfm.predict(X_train)
#     yhte= rfm.predict(X_test)
#     yhtr=rfm.predict(X_train)
#     r2_tr_list.append(r2_score(y_train,yhtr))
#     r2_te_list.append(r2_score(y_test,yhte))
#     mse_tr.append(mean_squared_error(y_train,yhtr))
#     mse_te.append(mean_squared_error(y_test,yhte))

In [6]:
"' N time step ahead random shuffle'"
step_list=list(range(1,96))
step_list[0]=1
r2_te_list=[]
r2_tr_list=[]
mse_tr=[]
mse_te=[]
var_im=[]
pmut_im=[]
y_te_pred_all=[]
y_te_all=[]

x_tr_all=[]
x_te_all=[]
y_tr_all=[]
for step in step_list:
    # step=1
    Cl_xstep1=Cl_x[:len(Cl_x)-step]
    Cl_ystep1=Cl_y[step:]
    X_train,X_test,y_train,y_test=train_test_split(Cl_xstep1,Cl_ystep1 ,test_size=0.2, random_state=random_seed)
    rfm = RandomForestRegressor(n_jobs=-1,n_estimators=800,random_state=random_seed,max_features=3,min_samples_leaf=20)
    rfm.fit(X_train,y_train)
    vari_imp_mn=rfm.feature_importances_
    # hpa_yh= rfm.predict(X_train)
    yhte= rfm.predict(X_test)
    # yhtr=rfm.predict(X_train)
    # plot_vari_imp=rfm.feature_importances_
    # resultss = permutation_importance(rfm, X_train, y_train, n_repeats=10,
    #                             random_state=0)
    y_te_pred_all.append(yhte)
    x_tr_all.append(X_train)
    y_tr_all.append(y_train)
    x_te_all.append(X_test)
    y_te_all.append(y_test)
    # pmut_im.append(resultss.importances_mean)
    # r2_tr_list.append(r2_score(y_train,yhtr))
    # r2_te_list.append(r2_score(y_test,yhte))
    # mse_tr.append(mean_squared_error(y_train,yhtr))
    # mse_te.append(mean_squared_error(y_test,yhte))
    # var_im.append(plot_vari_imp)

In [13]:
len(y_te_pred_all)

95

In [16]:
for i in range (len(y_te_pred_all)):
    x_tr_all[i]['Chlorine']=y_tr_all[i]
    x_te_all[i]['Chlorine']=y_te_all[i]
    x_te_all[i]['Chlorine_pred']=y_te_pred_all[i]
    np.savez('D:/istb_4/data/rf_regression/step_'+ str(i+1)+'.npz',te=x_te_all[i],tr=x_tr_all[i])

In [20]:
df['Time'].to_csv('D:/istb_4/data/rf_regression/time_index.csv')

In [21]:
x_te_all[i]

,pH,Conductivity.uS.,Temperature.C.,ORP.mV.,ISTB4.Sum,Chlorine,Chlorine_pred
12386,8.07,993.1,28.37,504.1,84,0.17,0.195629
13467,8.22,1091.0,26.65,508.5,29,NaN,0.191511
53181,7.83,1312.0,28.04,729.4,10,NaN,0.190285
45823,7.99,719.8,27.68,558.0,4,NaN,0.170527
49227,8.03,1047.0,28.58,622.0,4,NaN,0.188138
...,...,...,...,...,...,...,...
3727,7.95,940.3,26.86,429.9,18,0.31,0.306007
28136,8.32,1514.0,24.91,664.0,31,NaN,0.189216
29847,8.37,1022.0,24.85,640.0,17,NaN,0.257081
40011,7.78,1310.0,26.37,265.0,7,NaN,0.268165


In [ ]:
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_mse_tr_list_8hr.npy',mse_tr)
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_mse_te_list_8hr.npy',mse_te)
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_r2_tr_list_8hr.npy',r2_tr_list)
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_r2_te_list_8hr.npy',r2_te_list)
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_step_list_8hr.npy',step_list)
np.save('D:/istb_4/rf_regression/step_ahead_random_shuffle_RFR_varimp8hr.npy',var_im)


In [ ]:
np.save('D:/istb_4/rf_regression/varimp_permutation.npy',pmut_im)


In [ ]:
mse_tr=np.load('D:/istb_4/random_forest/step_ahead_random_shuffle_mse_tr_list_8hr.npy')
mse_te=np.load('D:/istb_4/random_forest/step_ahead_random_shuffle_mse_te_list_8hr.npy')
r2_tr_list=np.load('D:/istb_4/random_forest/step_ahead_random_shuffle_r2_tr_list_8hr.npy')
r2_te_list=np.load('D:/istb_4/random_forest/step_ahead_random_shuffle_r2_te_list_8hr.npy')
step_list=np.load('D:/istb_4/random_forest/step_ahead_random_shuffle_step_list_8hr.npy')

In [ ]:

txt_size=13
label_size=10
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(4,3), dpi=200)
ax.plot(np.array(step_list)*5/60,np.array(mse_tr)**0.5, 'r',markersize=3,label='RF regression Train')
ax.plot(np.array(step_list)*5/60,np.array(mse_te)**0.5,'b', marker='^', markersize=3,label='RF regression Test')
# ax.set_ylim([80,100])
ax.legend()
ax.set_xlabel('Time ahead (hr)',fontsize=txt_size)
ax.set_ylabel('$RMSE$ (ppm) ',fontsize=txt_size)
ax.tick_params(axis='both', direction='in',labelsize=label_size) 
plt.show()

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(4,3), dpi=200)
ax.plot(np.array(step_list)*5/60,r2_tr_list, 'r',markersize=3,label='RF regression Train')
ax.plot(np.array(step_list)*5/60,r2_te_list,'k', marker='^', markersize=3,label='RF regression test')
# ax.set_ylim([80,100])
# ax.legend()
ax.set_xlabel('Time ahead (hr)',fontsize=txt_size)
ax.set_ylabel('$R^2$ ',fontsize=txt_size)
ax.tick_params(axis='both', direction='in',labelsize=label_size) 
plt.show()

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Times']})
# rc.rcParams['font.size'] = 12
rc('text', usetex=True)
txt_size=13
label_size=10
mean_var_im=np.array(pmut_im).mean(axis=0)
variables_name=['pH', 'Conductivity', 'Temperature', 'ORP', 'Occupancy']

varplot=pd.DataFrame(data= mean_var_im, index=variables_name, columns=['gini'])
dfx = varplot.sort_values('gini',ascending=False)
fig, ax=plt.subplots(nrows=1, ncols=1, sharey= True, figsize=(5,3), dpi=200)

ax.bar(dfx.index,dfx['gini']/dfx['gini'].sum())
ax.set_ylabel('Variable importance',fontsize=txt_size)
ax.xaxis.set_ticks_position('both')
ax.tick_params(direction='in',labelsize=label_size)
ax.yaxis.set_ticks_position('both')
# ax.set_xticks(variables_name, fontsize=label_size)
plt.show()
